## Libraries

In [ ]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [1]:
import os
import tensorflow as tf
import pickle
import math
import pandas as pd
import numpy as np
import zipfile
from __future__ import division
import nltk
#nltk.download()
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to
       |     /mnt/home/divi/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/cess_cat

       | Downloading package twitter_samples to
       |     /mnt/home/divi/nltk_data...
       |   Unzipping corpora/twitter_samples.zip.
       | Downloading package udhr to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/udhr.zip.
       | Downloading package udhr2 to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/udhr2.zip.
       | Downloading package unicode_samples to
       |     /mnt/home/divi/nltk_data...
       |   Unzipping corpora/unicode_samples.zip.
       | Downloading package universal_treebanks_v20 to
       |     /mnt/home/divi/nltk_data...
       | Downloading package verbnet to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/verbnet.zip.
       | Downloading package verbnet3 to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/verbnet3.zip.
       | Downloading package webtext to /mnt/home/divi/nltk_data...
       |   Unzipping corpora/webtext.zip.
       | Downloading package wordnet to /mnt/home/divi/nltk_data...
       |   Unzi

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [3]:
# drive.mount('/content/gdrive/')

In [4]:
# !ls 'gdrive/My Drive/Microsoft AI'

## Loading Data

In [13]:
archive = zipfile.ZipFile('data.zip', 'r')
data = pd.read_csv(archive.open("data.tsv"),delimiter='\t',header=None)
data.columns = ['queryID','query','response','label','labelID']
data.isnull().values.any()

False

In [15]:
len(data)

5241880

In [16]:
# sample = data.iloc[:500].copy(deep = True)
# sample.to_csv('gdrive/My Drive/Microsoft AI/Sample_Raw_Data.csv',index=False)
sample = pd.read_csv('Clean_data_latest.csv')
sample.head()

,0,1,2,3,4
0,131,. corporation ?,"company incorporated specific nation , often w...",0,0
1,131,. corporation ?,"today , growing community 2,100 certified b co...",0,1
2,131,. corporation ?,"corporation definition , association individua...",0,2
3,131,. corporation ?,example corporation sentence . 1 work consulta...,0,3
4,131,. corporation ?,1 : government-owned corporation ( utility rai...,0,4


In [17]:
len(sample)

5241830

In [0]:
df1 = data.iloc[:2620940].copy(deep = True)
df2 = data.iloc[2620940:].copy(deep = True)

In [0]:
#data.head()

In [0]:
# sample = data.iloc[:500].copy(deep = True)
# sample.to_csv('gdrive/My Drive/Microsoft AI/Sample_Raw_Data.csv',index=False)
sample = pd.read_csv('gdrive/My Drive/Microsoft AI/Sample_Raw_Data.csv')
sample.head()

,queryID,query,response,label,labelID
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


## Cleaning

In [8]:
temp = data.copy(deep = True)
temp.head()

,queryID,query,response,label,labelID
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4


In [9]:
def clean(df,column):
  #Lowercase conversion
  df[column] = df[column].apply(lambda x: " ".join(x.lower() for x in x.split()))
  print(column+": Converted to lowercase")
  #Tokenization
  df[column] = df[column].apply(word_tokenize)
  print(column+": Tokenized")
  #Removing Stop Words
  #df[column] = [w for w in word_tokens if not w in stop_words]
  #df[column] = df[column].apply(lambda x: [item for item in x if item not in stop_words])
  df[column] = df[column].apply(lambda x: " ".join(word for word in x if word not in stop_words))
  print(column+": StopWords Removed")
  #Lemmatization - root words
  df[column] = df[column].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
  print(column+": Root words Lemmatized")
  return df[column]#.apply(word_tokenize)

  

In [10]:
temp['query']= clean(temp,'query')
temp['response'] = clean(temp,'response')
temp.head()

query: Converted to lowercase
query: Tokenized
query: StopWords Removed
query: Root words Lemmatized
response: Converted to lowercase
response: Tokenized
response: StopWords Removed
response: Root words Lemmatized


,queryID,query,response,label,labelID
0,131,. corporation ?,"company incorporated specific nation , often w...",0,0
1,131,. corporation ?,"today , growing community 2,100 certified b co...",0,1
2,131,. corporation ?,"corporation definition , association individua...",0,2
3,131,. corporation ?,example corporation sentence . 1 work consulta...,0,3
4,131,. corporation ?,1 : government-owned corporation ( utility rai...,0,4


In [0]:
print(sample['response'][0] ,'\n', temp['response'][0])

A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly. 
 company incorporated specific nation , often within bound smaller subset nation , state province . corporation governed law incorporation state . corporation may issue stock , either private public , may classified non-stock corporation . stock issued , corporation usually governed shareholder , either directly indirectly .


In [11]:
temp.to_csv("Clean_data_latest.tsv",sep='\t', index=False,header=None)
temp.to_csv("Clean_data_latest.csv", index=False,header=None)

In [12]:
len(temp)

5241880

## Feature Extraction

In [0]:
tempf = temp.copy(deep = True)
tempf.head(10)

,queryID,query,response,label,labelID
0,131,. corporation ?,"company incorporated specific nation , often w...",0,0
1,131,. corporation ?,"today , growing community 2,100 certified b co...",0,1
2,131,. corporation ?,"corporation definition , association individua...",0,2
3,131,. corporation ?,example corporation sentence . 1 work consulta...,0,3
4,131,. corporation ?,1 : government-owned corporation ( utility rai...,0,4
5,131,. corporation ?,mcdonald 's corporation one recognizable corpo...,1,5
6,131,. corporation ?,corporation owned stockholder ( shareholder ) ...,0,6
7,131,. corporation ?,association organized group people share commo...,0,7
8,131,. corporation ?,b corp certification shine light company leadi...,0,8
9,131,. corporation ?,llcs offer greater flexibility come income tax...,0,9


In [0]:
co = pd.read_csv("gdrive/My Drive/Microsoft AI/Data/submission_answer.tsv",delimiter='\t',header=None)
co.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1135787,17.352767,12.778355,10.567963,17.590332,7.879177,7.288547,2.737757,3.989637,4.548654,12.860450
1,281922,3.825710,6.209002,5.246799,2.336146,0.000000,0.000000,0.000000,2.658957,0.000000,0.000000
2,120233,9.169234,0.000000,8.920807,0.000000,0.000000,6.718768,5.173019,8.437319,5.022307,9.674624
3,319757,0.000000,0.000000,0.000000,0.000000,2.568677,0.000000,0.000000,3.094255,0.000000,0.000000
4,193633,0.000000,0.000000,3.909650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,50229,7.685039,0.000000,6.015412,5.503310,3.918687,0.000000,8.957478,4.974003,0.000000,0.000000
6,130810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.170028,0.000000
7,138208,5.914187,7.451903,7.514478,2.963889,10.851555,9.207414,6.086954,8.906415,6.455549,6.682578
8,334554,0.000000,0.000000,0.000000,0.000000,0.000000,3.891863,0.000000,0.000000,5.139703,0.000000
9,204203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [0]:
len(co)

10417

In [0]:
an = pd.read_csv("gdrive/My Drive/Microsoft AI/Data/answer.tsv",delimiter='\t',header=None)
an.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1135787,27.367275,36.495855,32.915477,30.555301,13.866893,26.627313,16.511408,8.397489,17.608676,36.688660
1,281922,11.744009,11.575771,8.146057,2.169596,1.034630,13.514372,2.190186,5.789057,10.278486,9.164840
2,120233,14.281651,0.000000,23.911319,0.000000,10.886682,10.715390,8.659164,20.936059,8.447720,14.982256
3,319757,-0.158872,-0.474097,6.921238,7.171402,11.052829,9.210453,19.166688,12.897403,7.666373,-0.472748
4,193633,14.829290,0.000000,21.061208,9.554759,7.337612,11.054364,12.213071,21.539519,9.041871,0.000000
5,50229,21.427538,3.578754,25.514792,28.052387,21.650233,11.975166,22.662141,16.703877,10.587768,11.500565
6,130810,10.302322,10.982077,0.483935,14.947725,0.728160,0.000000,16.132513,0.541912,4.051781,13.366268
7,138208,6.312909,9.123455,8.563088,3.501763,13.356172,11.197807,6.877595,10.856327,7.652591,7.576421
8,334554,0.000000,11.326957,6.975564,11.078372,7.559637,4.457489,13.616845,0.000000,15.751821,8.652441
9,204203,0.000000,0.000000,0.000000,17.796476,0.000000,22.761483,0.000000,12.312443,0.000000,15.289334


In [0]:
len(an)

10417

In [0]:
# ev = pd.read_csv("gdrive/My Drive/Microsoft AI/Data/eval1_unlabelled.tsv",delimiter='\t',header=None)
# ev.head(10)
# co.to_csv("gdrive/My Drive/Microsoft AI/Data/data_test.tsv", sep='\t', index=False,header=None )

In [0]:
# co = pd.read_csv("gdrive/My Drive/Microsoft AI/Data/data_test.tsv",delimiter='\t',header=None)
# co.head(10)

### BM25 -TFID

Save cleaned file as tsv and run BM-25.ipynb to get answers.tsv which is the BM-25 feature

In [0]:
with open('gdrive/My Drive/Microsoft AI/Data/docIDFDict.pickle', 'rb') as f:
  docIDFDict  = pickle.load(f)

avgDocLength = 56.337881256343145

#The following GetBM25Score method will take Query and passage as input and outputs their similarity score based on the term frequency(TF) and IDF values.
def GetBM25Score(Query, Passage, k1=1.5, b=0.75, delimiter=' ') :
    
    #global docIDFDict,avgDocLength

    query_words= Query.strip().lower().split(delimiter)
    passage_words = Passage.strip().lower().split(delimiter)
    passageLen = len(passage_words)
    docTF = {}
    for word in set(query_words):   #Find Term Frequency of all query unique words
        docTF[word] = passage_words.count(word)
    commonWords = set(query_words) & set(passage_words)
    tmp_score = []
    for word in commonWords :   
        numer = (docTF[word] * (k1+1))   #Numerator part of BM25 Formula
        denom = ((docTF[word]) + k1*(1 - b + b*passageLen/avgDocLength)) #Denominator part of BM25 Formula 
        if(word in docIDFDict) :
            tmp_score.append(docIDFDict[word] * numer / denom)

    score = sum(tmp_score)
    return score

In [0]:
for i in range(10):
  print( i,GetBM25Score(tempf['query'][i],tempf['response'][i]))

0 20.86015288831061
1 6.28060735050483
2 16.84416051497076
3 22.14220213166373
4 12.032298052844576
5 20.467823037536004
6 12.517690030542287
7 9.533295211588246
8 0
9 7.8631336178339994


In [0]:
co = pd.read_csv("gdrive/My Drive/Microsoft AI/Data/answer.tsv",delimiter='\t',header=None)
co.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,131,20.860153,6.280607,16.844161,22.142202,12.032298,20.467823,12.517690,9.533295,0.000000,7.863134
1,226969,13.013185,15.959825,10.611629,15.769753,7.186772,21.399558,15.619361,7.550264,16.690253,9.697560
2,18122,25.812820,39.057990,30.549143,17.960894,16.593171,36.159711,16.236718,28.926180,25.919034,21.841173
3,35300,29.032277,22.821485,17.855621,11.656895,20.690911,21.818315,26.443924,0.000000,23.372704,26.890020
4,90447,24.525248,24.525248,22.668690,5.195704,17.456777,0.000000,0.000000,20.607037,0.000000,0.000000
5,226977,30.273876,0.000000,32.206855,35.426650,14.980646,22.086062,12.911696,13.435371,29.333579,21.987032
6,154713,14.921140,13.834921,10.621379,10.811288,0.000000,17.782968,11.008112,0.000000,3.913028,17.111956
7,139430,31.128784,55.366975,58.696991,27.378438,55.813671,32.267185,56.892025,49.489101,32.119828,63.629769
8,57679,15.305510,0.000000,0.000000,11.196903,13.635169,14.775157,14.775157,10.488667,13.324114,15.791364
9,39439,26.583603,20.654649,0.000000,0.000000,14.878544,4.343485,17.191411,24.701346,17.859964,4.302454


In [0]:
q1 = tempf['query'][0]
q1

'. corporation ?'

In [0]:
a1 = tempf['response'][0]
a1

'company incorporated specific nation , often within bound smaller subset nation , state province . corporation governed law incorporation state . corporation may issue stock , either private public , may classified non-stock corporation . stock issued , corporation usually governed shareholder , either directly indirectly .'

### Named Entity Recognition

In [0]:
# from nltk.tag import StanfordNERTagger
# st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz','/usr/share/stanford-ner/stanford-ner.jar',encoding='utf-8')

In [0]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
#https://spacy.io/api/annotation#section-named-entities

In [0]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


In [0]:
doc.ents[1].label_

'ORG'

In [0]:
for i in range(len(doc.ents)):
  print(doc.ents[i].text,doc.ents[i].label_)

European NORP
Google ORG
$5.1 billion MONEY
Wednesday DATE


In [0]:
def NER(df):
  for i in tempf.index:
    qdoc = nlp(tempf['query'][i])
    
  